## Data

We apply the parametric portfolio policies developed by Brandt, Santa-Clara, and Valkanov (2009). The data file contains monthly returns for nine firm-specific characteristics: Market, SMB, HML, RMW, CMA, UMD, ROE, IA, BAB.
- Assume that these returns were generated by Nt stocks and that the number of stocks is constant over time.
- The first five characteristics (Market, SMB, HML, RMW, CMA) are from Fama and French (2015), the sixth (UMD) is from Carhart (1997), the profitability (ROE) and investment (IA) factors are from Hou, Xue, and Zhang (2015), and the betting-against-beta (BAB) factor is from Frazzini and Pedersen (2014).
- All factors are returns in excess of the risk-free rate.
- In particular, every factor (besides MKT and BAB) is the return of a long-short portfolio of stocks with 1usd in the long leg and 1usd in the short leg, and thus, their returns equal their excess returns.
- The MKT and BAB factors are also long-short portfolios because they are returns in excess of the risk-free rate.

## Parametric portfolio

Parametric portfolio policies show us how to reduce the dimensionality of the portfolio problem when investing in factors.
Instead of modeling asset returns in terms of factors, and then constructing the optimal portfolio, parametric portfolio policies directly specify portfolio weights in terms of factors.

$$w_{t}(\theta) = w_{b,t} + (F_{1,t} \theta_{1} + F_{2,t} \theta_{2} + ... + F_{K,t} \theta_{K})/N_{t}$$

where $w_{b,t}$ is the weight of the benchmark portfolio, $F_{k,t}$ is the factor k's return at time t and $N_{t}$ is the number of assets.

**Finding the optimal $\theta$ vector for a mean-variance investor with risk aversion of $\gamma=5$, assuming the investor can invest in only these nine factors. (No out-of-sample analysis).**

First, let's import the libraries we need for this problem

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

Then we load the given data set. Since we consider the market factor as the benchmark weights, we will allocate the returns of this factor into the excess return benchmark, and the other 8 factors into the excess returns Parametric Portfolio.

In [2]:
excess_returns = pd.read_excel('QPM-FactorsData.xlsx', index_col=[0])
excess_returns

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
196702,0.0078,0.0334,-0.0217,0.0194,-0.0094,0.0356,0.035317,-0.002064,0.0262
196703,0.0399,0.0163,0.0031,0.0090,-0.0151,0.0142,0.018876,-0.016933,0.0081
196704,0.0389,0.0062,-0.0264,0.0243,-0.0375,0.0064,0.010983,-0.029519,0.0171
196705,-0.0433,0.0198,0.0080,-0.0175,0.0161,0.0067,0.005234,0.024686,0.0201
196706,0.0241,0.0596,0.0096,-0.0064,-0.0239,0.0603,0.002945,-0.021700,-0.0163
...,...,...,...,...,...,...,...,...,...
202008,0.0763,-0.0094,-0.0294,0.0427,-0.0144,0.0051,-0.008682,-0.028534,-0.0399
202009,-0.0363,0.0007,-0.0251,-0.0115,-0.0177,0.0305,0.012255,-0.021992,0.0129
202010,-0.0210,0.0476,0.0403,-0.0060,-0.0053,-0.0303,-0.024671,-0.007387,-0.0201


In [3]:
excess_returns_bm = excess_returns['Market']
excess_returns_PP = excess_returns.iloc[:,1:]
excess_returns_PP

,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,
196702,0.0334,-0.0217,0.0194,-0.0094,0.0356,0.035317,-0.002064,0.0262
196703,0.0163,0.0031,0.0090,-0.0151,0.0142,0.018876,-0.016933,0.0081
196704,0.0062,-0.0264,0.0243,-0.0375,0.0064,0.010983,-0.029519,0.0171
196705,0.0198,0.0080,-0.0175,0.0161,0.0067,0.005234,0.024686,0.0201
196706,0.0596,0.0096,-0.0064,-0.0239,0.0603,0.002945,-0.021700,-0.0163
...,...,...,...,...,...,...,...,...
202008,-0.0094,-0.0294,0.0427,-0.0144,0.0051,-0.008682,-0.028534,-0.0399
202009,0.0007,-0.0251,-0.0115,-0.0177,0.0305,0.012255,-0.021992,0.0129
202010,0.0476,0.0403,-0.0060,-0.0053,-0.0303,-0.024671,-0.007387,-0.0201


Next, let's define a function ```MVP_weights``` to return objective function of the optimization problem and optimal weight for mean-variance portfolio.

In [4]:
# Define the objective function to be minimized (portfolio variance)
def obj_MVP(weights,bm_return,char_return):
    portfolio_return = bm_return + char_return @ weights
    mean_return = np.mean(portfolio_return)
    portfolio_risk = np.var(portfolio_return)
    return (gamma/2) * portfolio_risk - mean_return

# Design a function to calculate the optimal weight of GMV-C portfolio:
def MVP_C_w(bm_return,char_return):

    # Define the initial guess for weights
    initial_weights = np.ones(len(char_return.columns)) / (len(char_return.columns))

    # Use scipy.optimize.minimize to find the optimal weights
    result = minimize(obj_MVP, initial_weights, args=(bm_return,char_return,), method='SLSQP')
    
    # Extract the optimal weights
    optimal_weights = result.x
    
    return optimal_weights

Now let's calculate our theta for 8 factors

In [5]:
# Given value of risk tolerance
gamma = 5

# Calculate theta vector
theta = MVP_C_w(excess_returns_bm,excess_returns_PP)
theta

array([ 1.01658799, -0.136824  ,  0.54525987,  1.07803223,  0.20377353,
        1.71734256,  1.61456511,  0.72318278])

**Calculation of the Sharpe ratio for each of the nine factors and comparison to that of the parametric portfolio.**

First, let's calculate the Sharpe Ratio for each factor's returns

In [6]:
SR_factor = (excess_returns.mean() * 12) / (excess_returns.std() * np.sqrt(12))
SR_factor

Market    0.432904
SMB       0.197861
HML       0.277910
RMW       0.420687
CMA       0.477857
UMD       0.507284
ROE       0.684304
IA        0.626422
BAB       0.899567
dtype: float64

Now we calculate the Sharpe Ratio for the Parametric portfolio

In [7]:
# Vector of weights of 9 characteristics in the Parametric Portfolio
PP_weight = np.insert(theta, 0 ,1)

# Calculate Sharpe Ratio for Parametric Portfolio
PP_returns = excess_returns @ PP_weight
SR_PP = (np.mean(PP_returns) * 12) / (np.std(PP_returns) * np.sqrt(12))
SR_PP

1.4388440549546873

We can see that the Sharpe Ratio of the parametric portfolio outperforms all the Sharpe Ratios of individual factor portfolios.

## Volatility-managed Portfolio

Now we build on parametric portfolio policies, by modeling the portfolio weight on each factor, $θ_{k}$, to depend on factor or market volatility

We use an estimation window of 120 months. Therefore, to facilitate comparison, the in-sample and out-of-sample performance will be evaluated from January 1977 to December 2020 (monthly daily instead of daily data as in the original papers).

Define $f_{t+1}$ to be an excess return.

Construct a new volatility-managed factor, whose return is:

$$f_{t+1}^{\sigma} = \frac{c}{\sigma_t^2 (f)} * f_{t+1}$$ 

where:
- $\sigma_t (f)$ is the previous 12 month's realized volatility, estimated using monthly data
- choose $c$ so $f^{\sigma}$ has the same unconditional volatility as $f$

In [8]:
original_factors = pd.read_excel('QPM-FactorsData.xlsx', index_col=[0])
original_factors

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
196702,0.0078,0.0334,-0.0217,0.0194,-0.0094,0.0356,0.035317,-0.002064,0.0262
196703,0.0399,0.0163,0.0031,0.0090,-0.0151,0.0142,0.018876,-0.016933,0.0081
196704,0.0389,0.0062,-0.0264,0.0243,-0.0375,0.0064,0.010983,-0.029519,0.0171
196705,-0.0433,0.0198,0.0080,-0.0175,0.0161,0.0067,0.005234,0.024686,0.0201
196706,0.0241,0.0596,0.0096,-0.0064,-0.0239,0.0603,0.002945,-0.021700,-0.0163
...,...,...,...,...,...,...,...,...,...
202008,0.0763,-0.0094,-0.0294,0.0427,-0.0144,0.0051,-0.008682,-0.028534,-0.0399
202009,-0.0363,0.0007,-0.0251,-0.0115,-0.0177,0.0305,0.012255,-0.021992,0.0129
202010,-0.0210,0.0476,0.0403,-0.0060,-0.0053,-0.0303,-0.024671,-0.007387,-0.0201


Next we calculate the previous 12-months realized variance of each factor $\sigma_{t}^2 (f)$.

In [9]:
realized_variance = original_factors.rolling(12).var()
realized_variance = realized_variance[11:-1]
realized_variance

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
196801,0.001046,0.000371,0.000595,0.000346,0.000807,0.000795,0.000298,0.000611,0.000490
196802,0.001221,0.000618,0.000565,0.000312,0.000858,0.000917,0.000228,0.000677,0.000483
196803,0.001096,0.000713,0.000569,0.000317,0.000848,0.000965,0.000222,0.000661,0.000567
196804,0.001671,0.000816,0.000512,0.000335,0.000842,0.001113,0.000315,0.000642,0.000579
196805,0.001448,0.000957,0.000512,0.000306,0.000849,0.001158,0.000330,0.000639,0.000577
...,...,...,...,...,...,...,...,...,...
202007,0.004751,0.001022,0.002410,0.000181,0.000382,0.002688,0.000506,0.000393,0.002099
202008,0.004933,0.000999,0.002367,0.000328,0.000389,0.002256,0.000513,0.000442,0.002191
202009,0.005196,0.001005,0.001494,0.000320,0.000238,0.001766,0.000505,0.000328,0.002202


Finally we construct the volatility-managed factor by multiplying $\frac{c}{\sigma_{t}^2 (f)}$ by the monthly excess return of sed factor $f_{t+1}$. 

We set `c` as an average of the realized variance of the 9 original factors. Changing `c` won't impact the Sharpe Ratio but the leverage used in the portfolio.

In [10]:
c = realized_variance.mean().mean()
vol_managed = (c/realized_variance.values) * original_factors[12:]
vol_managed = vol_managed.rename(columns=lambda x: x + "_vol")
vol_managed

,Market_vol,SMB_vol,HML_vol,RMW_vol,CMA_vol,UMD_vol,ROE_vol,IA_vol,BAB_vol
Dates,,,,,,,,,
196802,-0.033684,-0.074830,0.018486,-0.003802,0.028999,-0.040310,-0.011930,0.042310,0.040892
196803,0.001540,-0.019471,-0.009808,0.035286,-0.012602,0.032589,0.069197,-0.015741,-0.030732
196804,0.077641,0.075525,-0.017011,0.082682,-0.040557,0.050188,0.159124,-0.037051,0.002818
196805,0.012827,0.074055,0.015422,0.010954,-0.020318,0.031496,0.058408,-0.033911,0.029373
196806,0.004479,-0.001670,0.012289,-0.040791,0.029688,-0.015587,-0.013588,0.035958,0.112239
...,...,...,...,...,...,...,...,...,...
202008,0.015096,-0.008646,-0.011468,0.222098,-0.035416,0.001784,-0.016131,-0.068272,-0.017867
202009,-0.006916,0.000658,-0.009969,-0.032935,-0.042777,0.012708,0.022433,-0.046807,0.005534
202010,-0.003799,0.044524,0.025357,-0.017602,-0.020915,-0.016130,-0.045884,-0.021148,-0.008581


Now we use mean-variance optimization to combine:
- The original (without timing) factor, $f_{t+1}$;
- The volatility-managed version of this factor, $f_{t+1}^{\sigma}$

First let's merge $f_{t+1}$ and $f_{t+1}^{\sigma}$ for each factor onto a separate dataframe.

In [11]:
factor_names = ["Market", "SMB", "HML", "RMW", "CMA", "UMD", "ROE", "IA", "BAB"]
factor_dfs = []

for i, factor_name in enumerate(factor_names):
    factor_df = pd.concat([original_factors.iloc[12:, i], vol_managed.iloc[:, i]], axis=1)
    factor_dfs.append(factor_df)

Market, SMB, HML, RMW, CMA, UMD, ROE, IA, BAB = factor_dfs

Let's create a function `mvo` to perform mean-variance optimization.

In [12]:
def mvo(dataframe):
    
    mu_sample = np.mean(dataframe, axis=0).values
    
    # Calculate inverse of covariance matrix
    sample_cov = dataframe.cov().values
    sample_Vinverse = np.linalg.inv(sample_cov)

    # Calculate the w_MVU
    w_MVU = (1/gamma) * sample_Vinverse @ mu_sample
    
    return w_MVU

Now we loop over each dataframe using the `mvo` function and a 120-month estimation window, to compute the weights for each window.

We then loop over again, this time to compute the out-of-sample $t+1$ monthly returns using the weights we just found. The returns are stored in a dictionary (with the key being the factor)

In [13]:
factor_returns = {} # Create an empty dictionary to store the returns for each factor

gamma = 5

for factor in factor_names:
    weights = []
    
    # Loop to calculate weights
    for i in range(120, len(eval(factor))):
        w_MVU = mvo(eval(factor)[i-120:i])
        weights.append(w_MVU)

    weights = np.array(weights)

    # Loop to calculate returns
    factor_returns[factor] = [] # Create an empty list for each factor
    for i in range(120, len(eval(factor))-1):
        factor_return = np.dot(weights[i-120].T, eval(factor).iloc[i])
        factor_returns[factor].append(factor_return)

    factor_returns[factor] = np.array(factor_returns[factor])

Let's compare the Sharpe ratios of:
- the portfolio with just the original factor and
- the portfolio that includes the volatility-timed factor.

In [14]:
def sharpe_ratio(returns):
    '''
    Function to return the Sharpe Ratio of a series of returns
    '''

    m_mean_return = returns.mean() # Monthly mean return
    m_vol = returns.std() # Monthly volatility
    
    y_mean_return = m_mean_return * 12 # Annualize the mean monthly return
    y_vol = m_vol * np.sqrt(12) # Transfer to annual volatility
    
    # Compute the Sharpe Ratio
    SR = y_mean_return / y_vol
    
    return SR


In [15]:
SR = {}

for factor in factor_names:
    SR[factor] = sharpe_ratio(factor_returns[factor])
    
SR = pd.DataFrame.from_dict(SR, orient='index', columns=['Sharpe Ratio Vol-Managed'])

SR

,Sharpe Ratio Vol-Managed
Market,0.368052
SMB,-0.159306
HML,0.564673
RMW,0.248971
CMA,0.237078
UMD,0.597577
ROE,0.940693
IA,0.544935
BAB,0.717256


In [16]:
original_SR = {}

for factor in factor_names:
    original_SR[factor] = sharpe_ratio(original_factors[132:][factor])
    
original_SR = pd.DataFrame.from_dict(original_SR, orient='index', columns=['Sharpe Ratio Original'])

# Concatenate the two sharpe ratio dataframes
SR_merged = pd.concat([SR, original_SR], axis = 1)

SR_merged

,Sharpe Ratio Vol-Managed,Sharpe Ratio Original
Market,0.368052,0.554375
SMB,-0.159306,0.168859
HML,0.564673,0.158015
RMW,0.248971,0.510641
CMA,0.237078,0.406058
UMD,0.597577,0.461568
ROE,0.940693,0.702961
IA,0.544935,0.506108
BAB,0.717256,0.867589


The portfolios including volatility-timed factors which have **higher** Sharpe Ratios than their original counterparts are: HML, UMD, ROE and IA.

The portfolios including volatility-timed factors which have **lower** Sharpe Ratios than their original counterparts are: Market, SMB, RMW, CMA and BAB.

Therefore, volatility scaling generates a higher Sharpe ratio for ONLY 4 out of the 9 factors we analyse.